In [1]:
### 합성곱 기반 문장 분류 모델 정의

import torch
from torch import nn


class SentenceClassifier(nn.Module):
    def __init__(self, pretrained_embedding, filter_sizes, max_length, dropout=0.5):
        super().__init__()

        # 학습된 임베딩 벡터
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(pretrained_embedding, dtype=torch.float32)
        )
        embedding_dim = self.embedding.weight.shape[1]

        conv = []
        for size in filter_sizes:
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels  = embedding_dim,
                        out_channels = 1,
                        kernel_size  = size
                    ),
                    nn.ReLU(),
                    ########################################원랜 -1 오타확인 (책 메모 참고)##################
                    nn.MaxPool1d(kernel_size=max_length-size+1),
                )
            )
        # Modulelist 클래스 : 여러 개의 서브 모듈을 리스트 형태로 저장
        self.conv_filters = nn.ModuleList(conv)

        output_size         = len(filter_sizes)
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout        = nn.Dropout(dropout)
        self.classifier     = nn.Linear(output_size, 1)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # 원본 텐서의 차원 순서가 [0, 1, 2] (즉, [배치 크기, 시퀀스 길이, 임베딩 차원])에서 
        # [0, 2, 1] (즉, [배치 크기, 임베딩 차원, 시퀀스 길이])로 변경되는 것을 의미 
        # 이러한 재배열은 일반적으로 1D 합성곱 연산을 수행하기 전에 필요한 데이터 형태 조정을 위해 사용됨
        embeddings = embeddings.permute(0, 2, 1)

        conv_outputs   = [conv(embeddings) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)

        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return logits

In [3]:
import pandas as pd
from Korpora import Korpora


corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/seoyun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/seoyun

In [4]:
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size : 45000
Testing Data Size : 5000


In [5]:
from konlpy.tag import Okt
from collections import Counter


def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab


tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


In [6]:
import numpy as np


def pad_sequences(sequences, max_length, pad_value):
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)


unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [8]:
from gensim.models import Word2Vec


n_vocab = len(token_to_id)
embedding_dim = 128


word2vec = Word2Vec.load("../models/word2vec.model")
init_embeddings = np.zeros((n_vocab, embedding_dim))

for index, token in id_to_token.items():
    if token not in ["<pad>", "<unk>"]:
        init_embeddings[index] = word2vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embeddings, dtype=torch.float32)
)

In [9]:
### 합성곱 신경망 분류 모델 학습

from torch import optim


device       = "cuda" if torch.cuda.is_available() else "cpu"
filter_sizes = [3, 3, 4, 4, 5, 5]
classifier   = SentenceClassifier(
    pretrained_embedding = init_embeddings,
    filter_sizes         = filter_sizes,
    max_length           = max_length
).to(device) 
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [10]:
def train(model, datasets, criterion, optimizer, device, interval): 
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets): 
        input_ids = input_ids.to(device)
        labels    = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss   = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


def test(model, datasets, criterion, device): 
    model.eval()
    losses   = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets): 
        input_ids = input_ids.to(device)
        labels    = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss   = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        # 분류 맞는지 확인
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


epochs   = 5
interval = 500

for epoch in range(epochs): 
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6801758408546448
Train Loss 500 : 0.5805291992699553
Train Loss 1000 : 0.5454219045577111
Train Loss 1500 : 0.5311488998702651
Train Loss 2000 : 0.5223988928403454
Train Loss 2500 : 0.5159138447937609
Val Loss : 0.46609184798150777, Val Accuracy : 0.7826
Train Loss 0 : 0.43668437004089355
Train Loss 500 : 0.48080227698037725
Train Loss 1000 : 0.4797555399643672
Train Loss 1500 : 0.4792746915212081
Train Loss 2000 : 0.47937491081882155
Train Loss 2500 : 0.47884384570408706
Val Loss : 0.45873842495508466, Val Accuracy : 0.7822
Train Loss 0 : 0.376381516456604
Train Loss 500 : 0.46664320099020673
Train Loss 1000 : 0.4715245616334778
Train Loss 1500 : 0.47281620994478285
Train Loss 2000 : 0.4689833408948602
Train Loss 2500 : 0.47034597847877335
Val Loss : 0.4486471884452497, Val Accuracy : 0.7872
Train Loss 0 : 0.34489211440086365
Train Loss 500 : 0.4596116668330933
Train Loss 1000 : 0.4643681386967639
Train Loss 1500 : 0.4633500731682317
Train Loss 2000 : 0.46549747681719